In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

In [2]:
events = pd.read_csv("events.csv.gzip", compression='gzip')

/home/cecilia/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
events.isnull().sum()

index                       0
date                        0
event_id                    0
ref_type                    0
ref_hash                    0
application_id              0
attributed                  0
device_countrycode          0
device_os_version     5411606
device_brand          5191157
device_model          2076489
device_city           5849646
session_user_agent      42280
trans_id              7706939
user_agent            4403098
event_uuid              29772
carrier               5818680
kind                    29772
device_os             5874391
wifi                        0
connection_type       5935285
ip_address                  0
device_language       2079172
dtype: int64

In [4]:
# Elimino las columnas que tienen mas de la mitad de los datos nulos
events = events.drop(['device_os_version','device_brand','device_city',
                           'trans_id','user_agent','carrier','device_os','connection_type'], axis=1)

In [14]:
events['event_id'].value_counts().head(10)

1      1281105
15     1005533
23      702708
2       621588
115     524729
0       404736
13      260978
7       256957
5       250572
101     218849
Name: event_id, dtype: int64

In [7]:
events['ref_type'].value_counts()

1891515180541284343    6421584
1494519392962156891    1322997
Name: ref_type, dtype: int64

In [15]:
events['application_id'].value_counts().head(10)

210    1900636
122    1021538
121     657386
65      654725
26      491855
269     276453
155     262531
21      257102
133     247082
226     222076
Name: application_id, dtype: int64

In [9]:
events['attributed'].value_counts()

False    7714809
True       29772
Name: attributed, dtype: int64

In [10]:
events['device_countrycode'].value_counts()

6287817205707153877    7744581
Name: device_countrycode, dtype: int64

In [16]:
events['kind'].value_counts().head(10)

4.017674e+18    1281105
5.500848e+18    1005533
6.081165e+18     683828
6.168309e+18     621588
6.123834e+18     524598
5.882882e+18     404733
4.853173e+18     260978
9.977660e+17     256493
9.208056e+18     250572
6.897685e+18     214713
Name: kind, dtype: int64

In [17]:
events['wifi'].value_counts()

False    5478103
True     2266478
Name: wifi, dtype: int64

In [23]:
events['device_language'].value_counts().head(10)

3.301378e+18    2588856
6.977049e+18    2311474
8.441417e+18     486372
6.584214e+18     101223
4.060930e+18      62155
5.221863e+18      52326
7.528974e+18      20169
3.491909e+18      14824
3.177265e+18       4668
4.077062e+17       1917
Name: device_language, dtype: int64

In [12]:
def crear_features_events(events_entrenar,datetime_inicio):
    events_entrenar['date'] = pd.to_datetime(events_entrenar['date'])
    
    dia_en_segundos = 86400
    events_entrenar['segundos'] = ((events_entrenar['date']-datetime_inicio).dt.seconds + 
                                     (events_entrenar['date']-datetime_inicio).dt.days*dia_en_segundos)

    
    # Cambio event_id por la probabilidad de que aparezca ese evento
    evento = events_entrenar['event_id'].value_counts(normalize=True)
    events_entrenar['event_id'] = events_entrenar['event_id'].replace(evento)
    
    # Cambio application_id por la probabilidad de que aparezca esa aplicacion
    aplicacion = events_entrenar['application_id'].value_counts(normalize=True)
    events_entrenar['application_id'] = events_entrenar['application_id'].replace(aplicacion)
      
    # Cambio kind por la probabilidad de que aparezca ese kind
    kind = events_entrenar['kind'].value_counts(normalize=True)
    events_entrenar['kind'] = events_entrenar['kind'].replace(kind)
    

    # Agrupo el data frame por los ref_hash
    events_entrenar = events_entrenar.groupby('ref_hash').agg(
        {'ref_hash':'count','event_id':'mean','application_id':'mean',
         'kind':'mean','segundos':'mean'})

    events_entrenar.columns = ['e_event_id','e_t_prom','e_kind','e_app_id','e_#eventos']
    events_entrenar = events_entrenar.reset_index('ref_hash')
    
    events_entrenar['e_eventoId'] = events_entrenar['e_event_id']*events_entrenar['e_#eventos']
    del events_entrenar['e_event_id']
    del events_entrenar['e_#eventos']
    
    return events_entrenar
    

In [13]:
events_entrenar = pd.read_csv('events_1820.csv')
datetime_inicio = datetime(2019,4,18)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train2_1820.csv', index=False)
f_events.head()

,ref_hash,e_t_prom,e_kind,e_app_id,e_eventoId
0,41863526108385,140062.647727,0.126640,0.211054,11.105743
1,161514654074162,147937.250000,0.057634,0.087007,0.470373
2,186034136943920,60125.769231,0.148482,0.253877,1.923081
3,360710529886978,214874.000000,0.064439,0.016072,0.197380
4,365882020742330,151254.982456,0.099017,0.019018,5.624120


In [14]:
events_entrenar = pd.read_csv('events_1921.csv')
datetime_inicio = datetime(2019,4,19)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train2_1921.csv', index=False)
f_events.head()

,ref_hash,e_t_prom,e_kind,e_app_id,e_eventoId
0,40621409780134,242807.833333,0.005631,0.005478,0.033673
1,41863526108385,88923.736111,0.147983,0.261183,10.618843
2,161514654074162,61537.250000,0.056457,0.083004,0.460309
3,360710529886978,155603.000000,0.070222,0.015924,0.286591
4,365882020742330,86141.076923,0.099317,0.019578,6.434779


In [15]:
events_entrenar = pd.read_csv('events_2022.csv')
datetime_inicio = datetime(2019,4,20)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train2_2022.csv', index=False)
f_events.head()

,ref_hash,e_t_prom,e_kind,e_app_id,e_eventoId
0,40621409780134,156407.833333,0.005677,0.005505,0.033952
1,41863526108385,31455.568627,0.146337,0.262990,7.438730
2,161514654074162,241464.294118,0.091154,0.088657,4.650622
3,168103949904656,173599.100000,0.090038,0.035244,0.919221
4,186034136943920,237222.571429,0.133386,0.262990,3.722590


In [16]:
events_entrenar = pd.read_csv('events_2123.csv')
datetime_inicio = datetime(2019,4,21)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train2_2123.csv', index=False)
f_events.head()

,ref_hash,e_t_prom,e_kind,e_app_id,e_eventoId
0,40621409780134,122304.666667,0.008657,0.005934,0.077695
1,41863526108385,9126.125000,0.085450,0.264696,0.681199
2,69039685746313,235984.500000,0.044512,0.028056,0.177472
3,161514654074162,165162.413793,0.093511,0.083326,5.427672
4,168103949904656,87199.100000,0.087588,0.034847,0.896247


In [17]:
events_entrenar = pd.read_csv('events_2426.csv')
datetime_inicio = datetime(2019,4,24)
f_events = crear_features_events(events_entrenar,datetime_inicio)
f_events.to_csv('events_train2_2426.csv', index=False)
f_events.head()

,ref_hash,e_t_prom,e_kind,e_app_id,e_eventoId
0,41863526108385,192838.719298,0.135702,0.222804,7.702390
1,69039685746313,229631.812500,0.061761,0.029999,0.984140
2,90072729247980,66661.000000,0.048354,0.222804,0.144452
3,161514654074162,154222.500000,0.084897,0.089999,0.174180
4,168103949904656,109329.666667,0.084897,0.032703,0.261271
